<DIV ALIGN=CENTER>

# Introduction to Machine Learning Pre-Processing
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore data pre-processing. This is a
large topic, so in this notebook we will focus on some basic concepts.
You should feel free to try completing additional tasks.

-----

